In [11]:
import pickle
import json
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.cm as cm
from six.moves import cPickle as pickle
from PIL import Image
import numpy as np

In [12]:
A = pickle.load(open('save/resnet50_mil_1000words/infos.pkl', 'rb'), encoding='iso-8859-1')

In [13]:
R = A['val_result_history']

In [14]:
P = R[468000]

In [15]:
P = P['predictions']

In [16]:
coco = json.load(open('data/coco/dataset_coco.json'))
coco = coco['images']

In [17]:
Caps = {}
for item in coco:
    Caps[item['cocoid']] = ' '.join(["<li>"+sent['raw']+"</li>" for sent in item['sentences']])

In [40]:
extra = pickle.load(open('mil_train_results.pkl', 'rb'))

In [41]:
Res = {}
for item in extra:
    Res[item['image_id']] = {
        'words': item['words'].split(),
        'probs': item['probs'],
        'regions': item['regions'],
        'region scores': item['region probs'],
        'region codes': item['region codes']
    }
    output = ''
    for w, p, r, pr in zip(item['words'].split(), item['probs'], item['regions'], item['region probs']):
        output += "<li>" + w + " - score: %.3f" % p + " - Region %d (%.3f)</li>" % (r , pr)
    Res[item['image_id']]['output'] = output

In [42]:
list(Res)[:10]

[57870, 384029, 222016, 520950, 69675, 547471, 122688, 392136, 398494, 90570]

In [ ]:
path = "file:////home/thoth/melbayad/scratch/work/torchcap/data/coco/images/val2014"
with open('mil.html', 'w') as f:
    f.write('<body topmargin="50" leftmargin="100" rightmargin="0">')
    for id in Res:
        f.write('<img style="width: 300px; height: 300px;" src="%s/COCO_val2014_%012d.jpg"> <br> \
                 <p> <b>Captions: </b> </p>\
                 <ul> %s </ul>\
                 <p> <b> MIL: </b> </p>\
                 <ul> %s </ul>\
                 <br>' % (path, id, Caps[id], Res[id]['output']))
    f.write('</body>')

In [ ]:
id = 403385
img = Image.open('data/coco/images/val2014/COCO_val2014_%012d.jpg' % id)
img = img.resize((300, 300), Image.ANTIALIAS)
fig, ax = plt.subplots(1, figsize=(10, 10))
# Display the image
ax.imshow(img)
# Create a Rectangle patch
colors = cm.rainbow(np.linspace(0, 1, 17))
visited = []
for e, r in enumerate(Res[id]['regions']):
    # from index find the corresponding receptive field
    x = 28 + (r % 9) * 32 - 106
    y = 28 + (r // 9) * 32 - 106
    w = x + 212
    h = y + 212
    x = max(x, 0)
    y = max(y, 0)
    print('r:', r, 'word:', Res[id]['words'][e], 'x: ', x, 'y:', y)
    rect = patches.Rectangle((x, y), w, h,
                             linewidth=1,
                             edgecolor=colors[e],
                             facecolor='none')
    if (x,y) not in visited:
        shift = 8
    else:
        shift = 17
    ax.text(x,y + shift,
        Res[id]['words'][e]+' ' +
        str(round(Res[id]['region scores'][e], 3)),
        fontsize=12, weight = 'bold', color = colors[e])
            
    # Add the patch to the Axes
    ax.add_patch(rect)
    visited.append((x,y))
    print('Visited: visited')
plt.show()

In [20]:
Res[403385]

{'output': '<li>toilet - score: 0.996 - Region 78 (0.867)</li><li>bathroom - score: 0.988 - Region 64 (0.428)</li><li>white - score: 0.947 - Region 75 (0.197)</li><li>small - score: 0.898 - Region 73 (0.161)</li><li>walls - score: 0.930 - Region 59 (0.331)</li><li>wall - score: 0.936 - Region 14 (0.263)</li><li>bath - score: 0.946 - Region 66 (0.623)</li><li>room - score: 0.946 - Region 7 (0.145)</li><li>public - score: 0.892 - Region 46 (0.293)</li><li>toilets - score: 0.864 - Region 70 (0.591)</li><li>sitting - score: 0.834 - Region 72 (0.164)</li><li>sink - score: 0.816 - Region 42 (0.130)</li><li>restroom - score: 0.824 - Region 71 (0.127)</li><li>tiled - score: 0.853 - Region 35 (0.157)</li><li>stall - score: 0.805 - Region 29 (0.085)</li><li>next - score: 0.759 - Region 0 (0.742)</li>',
 'probs': array([ 0.99596626,  0.98816633,  0.94667959,  0.89847183,  0.93025613,
         0.93566608,  0.94552279,  0.94555372,  0.89160329,  0.86436069,
         0.83357632,  0.81630826,  0.8235

In [43]:
# get word/region embeddings
Embed = {}
sum_scores = {}
cnt = 0
for id in Res:
    item = Res[id]
    scores = item['region scores']
    codes = item['region codes']
    words = item['words']
    for w, c, s in zip(words, codes, scores):
        cnt += 1
        if w not in Embed:
            Embed[w] = s * c
            sum_scores[w] = s
        else:
            Embed[w] += s * s
            sum_scores[w] += s

In [44]:
Embed['next']

array([ 55192.80859375,  55188.50390625,  55188.50390625,  55195.69921875,
        55188.50390625,  55190.01171875,  55191.02734375,  55190.21484375,
        55188.50390625,  55188.50390625,  55188.50390625,  55194.24609375,
        55192.51171875,  55188.50390625,  55188.50390625,  55188.50390625,
        55188.50390625,  55188.50390625,  55188.50390625,  55198.51171875,
        55191.01953125,  55194.87890625,  55188.50390625,  55203.05078125,
        55188.50390625,  55193.39453125,  55188.50390625,  55192.12890625,
        55188.50390625,  55188.50390625,  55188.50390625,  55193.19921875,
        55191.06640625,  55188.50390625,  55190.23828125,  55189.98046875,
        55188.50390625,  55189.68359375,  55189.58203125,  55188.50390625,
        55189.08984375,  55188.50390625,  55188.50390625,  55201.73046875,
        55195.56640625,  55188.50390625,  55188.50390625,  55188.56640625,
        55190.11328125,  55188.50390625,  55188.50390625,  55188.50390625,
        55188.50390625,  

In [45]:
len(Embed), len(sum_scores)
for id in Embed:
    Embed[id] /= sum_scores[id]

In [46]:

Embed['next']

array([ 0.88164026,  0.88157147,  0.88157147,  0.88168645,  0.88157147,
        0.88159561,  0.88161182,  0.88159883,  0.88157147,  0.88157147,
        0.88157147,  0.8816632 ,  0.88163555,  0.88157147,  0.88157147,
        0.88157147,  0.88157147,  0.88157147,  0.88157147,  0.88173133,
        0.8816117 ,  0.88167334,  0.88157147,  0.88180387,  0.88157147,
        0.88164961,  0.88157147,  0.88162941,  0.88157147,  0.88157147,
        0.88157147,  0.88164651,  0.88161242,  0.88157147,  0.88159919,
        0.88159508,  0.88157147,  0.88159037,  0.8815887 ,  0.88157147,
        0.88158083,  0.88157147,  0.88157147,  0.88178277,  0.8816843 ,
        0.88157147,  0.88157147,  0.88157248,  0.88159722,  0.88157147,
        0.88157147,  0.88157147,  0.88157147,  0.88161665,  0.88168168,
        0.88157147,  0.88157147,  0.88157147,  0.88162768,  0.88157147,
        0.88162667,  0.88160384,  0.88157147,  0.88170826,  0.88164026,
        0.88157147,  0.88157147,  0.88157147,  0.88166994,  0.88

In [47]:
pickle.dump(Embed, open('data/embeddings/regions.pkl', 'wb'))